# Week3_4 Assignment

## [BASIC](#Basic) 
- Encoder & Decoder Layer 코드를 직접 필사하고 각 함수에 주석을 달 수 있다. 

## [CHALLENGE](#Challenge)
- 텐서의 크기(shape)를 계산할 수 있다. 

## [ADVANCED](#Advanced)
- 완성된 transformer 모델의 모든 학습 가능한 파라미터 이름과 크기(shape)를 출력할 수 있다.

### Informs
이번 과제에서는 "[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)"의 코드를 필사해본다.   
"Annotated Transformer"는 "Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현한다.   
코드 필사를 통해 다음을 배울 수 있다.    
- Encoder, Decoder 구조
- Attention Mechanism
- "residual connection", "layer normalization" 등의 구조 

코드 필사를 시작하기 앞서, transformer 모델의 최종 구조를 살펴보자.    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_full.png?raw=true" width="500" align="center"/>

최종 모델은 `EncoderDecoder()` 클래스에 여러 인스턴스를 생성자의 입력 파라미터로 넣어 생성한다.    
앞으로 우리는 `EncoderDecoder()` 클래스와 같은 여러 클래스들을 구현하고 연결할 것이다. 따라서 대략적인 클래스간의 관계를 살펴보고 이해한다면 보다 큰 그림을 가지고 코드 필사를 할 수 있을 것이다. 

Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    
아래 좌측 도식에서 각 클래스의 색상은 아래 우측 도식(transformer 구조)의 색상과 맵핑되어 있다.    
각 클래스의 역할과 클래스 간 연결 관계를 생각하면서 transformer를 코드로 구현해보자.   


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_map.png?raw=true" width="400" height="400" align="left"/>
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="right"/>



In [ ]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

## Basic

### Frame
- `EncoderDecoder`

아래 도식은 `EncoderDecoder` 클래스의 `forward()`, `encode()`, `decode()` 메소드를 도식화 한 것이다.    
 
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_encoderdecoder.png?raw=true" width=500>


- `Generator`

In [ ]:
class EncoderDecoder(nn.Module):
    
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
      super(EncoderDecoder,self).__init__()
      self.encoder = encoder
      self.decoder = decoder
      self.src_embed = src_embed
      self.tgt_embed = tgt_embed
      self.generator = generator
    
    
    def forward(self, src, tgt, src_mask, tgt_mask):
      return self.decode(self.encode(src,src_mask),src_mask,tgt,tgt_mask) #encoder에서 출력된 임베딩 벡터를 디코더로 전송
    
    
    def encode(self, src, src_mask):
      return self.encoder(self.src_embed(src),src_mask) # 문장을 입력받아 생성된 임베딩벡터와 마스크 정보를 입력
    
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
      return self.decoder(self.tgt_embed(tgt),memory,src_mask,tgt_mask) # 인코더에서 처리된 정보를 문장2와 마스크정보를 받아 처리

In [ ]:
class Generator(nn.Module):
    
    def __init__(self, d_model, vocab):
      super(Generator,self).__init__()
      self.proj=nn.Linear(d_model,vocab) #트랜스포머 디코더 최종단의 단어 선택을 위한 NN모듈을 생성
    
    
    def forward(self, x):
      return F.log_softmax(self.proj(x),dim=1) #연산된 정보를 토대로 확율정보 출력

### Encoder
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`
- Reference
    - Layer Normalization
        - [한국어 설명](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)
        - [torch official docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
    - Residual Connection
        - [한국어 설명](https://itrepo.tistory.com/36)
    - pytorch ModuleList
        - [torch official docs](https://pytorch.org/docs/1.9.1/generated/torch.nn.ModuleList.html)


In [ ]:
def clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)]) #레이어 층쌓기

In [ ]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
      super(Encoder,self).__init__()
      self.layers = clones(layer,N) #입력값에 따른 레이어리스트 생성
      self.norm = LayerNorm(layer.size) #정규화를 위한 함수 생성
    
    
    def forward(self, x, mask):
      for layer in self.layers: #쌓인 레이어를 통한 연산 실시
        x = layer(x,mask)
      return self.norm(x) #정규화

In [ ]:
class LayerNorm(nn.Module):
    
    def __init__(self, features, eps=1e-6):
      super(LayerNorm,self).__init__()
      self.a_2 = nn.Parameter(torch.ones(features))
      self.b_2 = nn.Parameter(torch.zeros(features))
      self.eps = eps
    
    
    def forward(self, x):
      mean = x.mean(-1,keepdim=True)
      std = x.std(-1,keepdim=True)
      return self.a_2 * (x - mean) / (std + self.eps)+self.b_2 #평균과 표준편차를 이용한 정규화를 진행

In [ ]:
class SublayerConnection(nn.Module): #잔차연결 및 정규화
    
    def __init__(self, size, dropout):
      super(SublayerConnection,self).__init__()
      self.norm = LayerNorm(size) #정규화 클래스
      self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer): # x: 입력 임베딩벡터, sublayer: 서브레이어를 통과한 출력 입베딩벡터
      return x + self.dropout(sublayer(self.norm(x))) #출력된 임베딩 벡터를 정규화한 후 잔차연결

In [ ]:
class EncoderLayer(nn.Module):
    
    def __init__(self, size, self_attn, feed_forward, dropout):
      super(EncoderLayer,self).__init__()
      self.self_attn = self_attn
      self.feed_forward = feed_forward
      self.sublayer = clones(SublayerConnection(size,dropout),2) #잔차연결 및 정규화를 위한 클래스 생성(MultiHeadedAttention, PositionwiseFeedForward)
      self.size = size
        
    def forward(self, x, mask):
      x = self.sublayer[0](x, lambda x: self.self_attn(x,x,x,mask)) #하위 레이어 첫번째에 임베딩벡터 x를 전달 (K,Q,V의 3개 벡터를 생성하기 위해 동일값을 3번 전달)
      return self.sublayer[1](x,self.feed_forward) #하위 레이어 두번째에 첫번째 하위레이어의 값을 전달하여 신경망연산을 진행

### Decoder
- `Decoder`
- `DecoderLayer`

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self, layer, N):
      super(Decoder,self).__init__()
      self.layers = clones(layer,N) #입력값에 따른 레이어리스트 생성
      self.norm = LayerNorm(layer.size) #정규화 클래스
    
    def forward(self, x, memory, src_mask, tgt_mask):
      for layer in self.layers: #DecoderLayer로 생성된 레이어를 하나씩 진행하며 연산
        x = layer(x,memory,src_mask,tgt_mask) #문자열의 임베딩 벡터를 입력으로 encoder 의 입베딩 벡터를 K,V로 사용하여 어텐션을 진행 
      return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
      super(DecoderLayer, self).__init__()
      self.size = size
      self.self_attn = self_attn #셀프 어텐션을 진행하는 첫번째 하위 레이어
      self.src_attn = src_attn #encoder의 임베딩벡터를 입력받아 어텐션을 진행하는 두번째 하위 레이어
      self.feed_forward = feed_forward #신경망 연산을 진행하는 세번째 하위 레이어
      self.sublayer = clones(SublayerConnection(size, dropout), 3) #잔차연결 및 정규화를 위한 클래스 생성(MultiHeadedAttention,MultiHeadedAttention, PositionwiseFeedForward)
    
    
    def forward(self, x, memory, src_mask, tgt_mask):
      m = memory #encoder에서 연산된 최종 출력값
      x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask)) #첫번째 셀프어텐선을 진행(decoder로 입력된 문자열을 마스킹하여 처리)
      x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask)) # 이전 하위 레이어의 임베딩벡터를 Q, encoder의 임베딩벡터를 K,V로 하여 두번째 어텐션 진행
      return self.sublayer[2](x, self.feed_forward) #세번째 하위레이어로 신경망 연산을 진행

### Sublayer
- `attention` 함수

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_attention.png?raw=true" width="500" align="center"/>  

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

### Challenge


### Q1. 위 도식에 따라 `score`, `p_attn`, `attention` 을 구하라 

In [47]:
def attention(query, key, value, mask=None, dropout=None):
  d_k = query.size(-1) #d_model의 사이즈에서 h개의 차원으로 분할되어 각 어텐션의 차원길이를 나타냄
  scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k) #query,key의 dot연산을 진행후 스케일링(d_k의 루트값으로 다운 스케일)
  print('score:',scores.shape)
  if mask is not None:
      scores = scores.masked_fill(mask == 0, -1e9) #마스킹 존재시 0값을 아주작은 값으로 치환
  p_attn = F.softmax(scores, dim = -1) #어텐션 스코어를 통한 softmax의 확율값으로 변환
  print('p-attn:',p_attn.shape)
  if dropout is not None:
      p_attn = dropout(p_attn)
  return torch.matmul(p_attn, value), p_attn #어텐션스코어를 반영한 임베딩 벡터 및 어텐션 스코어 반환

In [53]:
d_model=512
query, key, value = [torch.ones((12,d_model)) for _ in range(3)]
print('shape:',query.shape)
print('attention:',attention(query, key, value)[0].shape)

shape: torch.Size([12, 512])
score: torch.Size([12, 12])
p-attn: torch.Size([12, 12])
attention: torch.Size([12, 512])


###Q2. query, key, value가 모두 (m, d_k) shape의 matrix라고 가정할 때, `score`, `p_attn`, `attention`의 shape을 각각 구하라
- score : (m, m)
- p_attn : (m, m)
- attention : (m, d_k)

### (아래의 **Q3을 먼저 풀고 돌아오세요**) Q4.  query, key, value가 모두 (12, 8, 1, 64) shape의 tensor라고 가정할 때 , `score`, `p_attn`, `attention`의 shape을 각각 구하라

- score : (12, 8, 1, 1)
- p_attn : (12, 8, 1, 1)
- attention : (12, 8, 1, 64)

- `MultiHeadedAttention`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
      super(MultiHeadedAttention, self).__init__()
      assert d_model % h == 0

      self.d_k = d_model // h # 임베딩백터의 차원 d_model을 h개로 나누어 h개의 어텐션을 생성했을 때의 각 벡터길이 
      self.h = h # 다중처리할 어텐션의 개수
      self.linears = clones(nn.Linear(d_model, d_model), 4) #임베딩 벡터를 이용한 Query,Key,Value 생성을 위한 신경망 리스트
      self.attn = None
      self.dropout = nn.Dropout(p=dropout)
    
    
    def forward(self, query, key, value, mask=None):
      if mask is not None:
          mask = mask.unsqueeze(1)
      nbatches = query.size(0)
      
      query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) #Q,K,V를 생성, d_model의 길이만큼 생성하여 h개로 나누어 각 어텐션으로 연산
                          for l, x in zip(self.linears, (query, key, value))]
      
      x, self.attn = attention(query, key, value, mask=mask,dropout=self.dropout) # 생성된 Q,K,V를 이용하여 어텐션 수행
      
      x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
      return self.linears[-1](x)
            

In [51]:
d_model=512
h=8
query, key, value = [torch.ones((12,d_model)) for _ in range(3)]
multiheadedattention=MultiHeadedAttention(h,d_model)
nbatches = query.size(0)

print('d_k:',multiheadedattention.d_k)
print('nn.Linear(d_model, d_model)(query):',nn.Linear(d_model, d_model)(query).shape)
print('nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k):',nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, multiheadedattention.d_k).shape)
print('nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2):',nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, multiheadedattention.d_k).transpose(1,2).shape)

d_k: 64
nn.Linear(d_model, d_model)(query): torch.Size([12, 512])
nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k): torch.Size([12, 1, 8, 64])
nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2): torch.Size([12, 8, 1, 64])


### Q3.  query, key, value가 모두 (12, 512) shape의 matrix이고, h 값이 8 이라고 가정할 때, 아래 값의 shape을 각각 구하라

- `d_k` (d_k = d_model // h) : 64
- `nn.Linear(d_model, d_model)(query)` : (12, 512)
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k)` : (12, 1, 8, 64) 
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2)` : (12, 8, 1, 64)

- `PositionwiseFeedForward`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pwff.png?raw=true" width="300" align="center"/>  

In [ ]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self, d_model, d_ff, dropout=0.1):
      super(PositionwiseFeedForward, self).__init__()
      self.w_1 = nn.Linear(d_model, d_ff)
      self.w_2 = nn.Linear(d_ff, d_model) #풀리커넥티드 네트워크로 d_ff의 크기의 노드 연산을 거친후 원래 d_model사이즈로 다시 압축
      self.dropout = nn.Dropout(dropout)

  def forward(self, x):
      return self.w_2(self.dropout(F.relu(self.w_1(x)))) # d_model -> d_ff -> d_model 사이즈로 노드 연산

### Input Embedding & Encoding
- `Embeddings`
    - [pytorch official docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [ ]:
class Embeddings(nn.Module):
  
    def __init__(self, d_model, vocab):
      super(Embeddings, self).__init__()
      self.lut = nn.Embedding(vocab, d_model) #총 단어 개수의 d_model길이의 임베딩 백터를 생성하는 모듈
      self.d_model = d_model #임베딩 벡터 길이
  
    def forward(self, x):
      return self.lut(x) * math.sqrt(self.d_model) #포지션 벡터를 합산시 해당 임베딩벡터의 의미가 희석되는 것을 보안하기 위해 증폭

- `PositionalEncoding`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  

- `position` 변수 설명
    - 모든 position (=최대 토큰 개수)의 값을 갖고 있는 matrix
- `div_term` 변수 설명

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_div.png?raw=true" width="500" align="center"/>  
- `Embedding` + `Encoding` 도식화 

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_emb_enc.png?raw=true" width="400" align="center"/>  


In [ ]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, dropout, max_len = 5000):
      super(PositionalEncoding, self).__init__()
      self.dropout = nn.Dropout(p=dropout)
      
      
      pe = torch.zeros(max_len, d_model)
      position = torch.arange(0, max_len).unsqueeze(1)
      div_term = torch.exp(torch.arange(0, d_model, 2) *
                            -(math.log(10000.0) / d_model))
      pe[:, 0::2] = torch.sin(position * div_term) #cos과 sin을 번갈아 가면서 position matrix를 생성
      pe[:, 1::2] = torch.cos(position * div_term)
      pe = pe.unsqueeze(0)
      self.register_buffer('pe', pe)
        
    def forward(self, x):
      x = x + Variable(self.pe[:, :x.size(1)], #역전파 되지 않도록 설정 후 임베딩 벡터와 합산
                        requires_grad=False)
      return self.dropout(x)

### Q4.  max_len이 512이고, d_model이 512라고 가정할 때, `position`과 `div_term`의 shape을 구하라

- `position` : (512,1)
- `div_term` : (256,)
- `position * div_term` : (512,256)

### Advanced

### Finally Build Model
- Xavier Initialization
    - [한국어 자료](https://huangdi.tistory.com/8)
    - [pytorch official docs](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_)

In [ ]:
def make_model(src_vocab, tgt_vocab, 
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
  c = copy.deepcopy
  attn = MultiHeadedAttention(h, d_model) # 하위 레이어 어텐션레이어 생성
  ff = PositionwiseFeedForward(d_model, d_ff, dropout) #하위 레이어 FFNN 레이어 생성
  position = PositionalEncoding(d_model, dropout) #포지션 정보를 생성
  model = EncoderDecoder(
      Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
      Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                            c(ff), dropout), N),
      nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
      nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
      Generator(d_model, tgt_vocab))
  
  for p in model.parameters(): #모델의 모든 파라미터값 초기화
      if p.dim() > 1:
          nn.init.xavier_uniform(p)
  return model
            

In [ ]:
model = make_model(10,10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


### Q5. 위 코드로 만든 모델의 모든 파라미터의 이름과 크기 (shape) 을 출력하라

In [ ]:
# 구현

for param in model.named_parameters():
  print('parameter name:',param[0])
  print('shape:',param[1].shape)

parameter name: encoder.layers.0.self_attn.linears.0.weight
shape: torch.Size([512, 512])
parameter name: encoder.layers.0.self_attn.linears.0.bias
shape: torch.Size([512])
parameter name: encoder.layers.0.self_attn.linears.1.weight
shape: torch.Size([512, 512])
parameter name: encoder.layers.0.self_attn.linears.1.bias
shape: torch.Size([512])
parameter name: encoder.layers.0.self_attn.linears.2.weight
shape: torch.Size([512, 512])
parameter name: encoder.layers.0.self_attn.linears.2.bias
shape: torch.Size([512])
parameter name: encoder.layers.0.self_attn.linears.3.weight
shape: torch.Size([512, 512])
parameter name: encoder.layers.0.self_attn.linears.3.bias
shape: torch.Size([512])
parameter name: encoder.layers.0.feed_forward.w_1.weight
shape: torch.Size([2048, 512])
parameter name: encoder.layers.0.feed_forward.w_1.bias
shape: torch.Size([2048])
parameter name: encoder.layers.0.feed_forward.w_2.weight
shape: torch.Size([512, 2048])
parameter name: encoder.layers.0.feed_forward.w_2.b